<a href="https://colab.research.google.com/github/zihadbappy/bnfake/blob/main/Final_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [288]:
!git clone https://github.com/zihadbappy/bnfakedata.git

fatal: destination path 'bnfakedata' already exists and is not an empty directory.


In [289]:
import pandas as pd
Dataset= pd.read_csv('/content/bnfakedata/bnfakefinal.csv')

In [290]:
Dataset.head()

,headline,content,label
0,সময় হইলেই ধরব কামাল,নারায়নগঞ্জের চাঞ্চল্যকর সাত খুন মামলায় অভিযুক্...,0
1,ওয়ান ষ্টপ সার্ভিস খুললেন বাবুনগরী,নিজস্ব মতিবেদক বিবিধ রাজনৈতিক সংকট সমাধানের পর...,0
2,কমিশনের হার বৃদ্ধি করবেন তারেক,লনডন মতিনিধি সংসদ নির্বাচনে জয় লাভ প্রধান মন্...,0
3,আর বাংলাদেশে ফুন দিব না বানকি মুন,জাতিসংঘ মতিবেদক ভবিষ্যতে বাংলাদেশে টেলিফোন অংগ...,0
4,ফোন না করায় জাতিসংঘের মহাসচিবের প্রতি ক্ষুব্ধ...,নিজস্ব মতিবেদক বাকশালের মহিলা আমীর প্রধান মন্ত...,0


In [291]:
df = pd.DataFrame(Dataset)

In [292]:
df = df.reset_index(drop=True)
# print(df)

In [293]:
df=df.dropna()
df

,headline,content,label
0,সময় হইলেই ধরব কামাল,নারায়নগঞ্জের চাঞ্চল্যকর সাত খুন মামলায় অভিযুক্...,0
1,ওয়ান ষ্টপ সার্ভিস খুললেন বাবুনগরী,নিজস্ব মতিবেদক বিবিধ রাজনৈতিক সংকট সমাধানের পর...,0
2,কমিশনের হার বৃদ্ধি করবেন তারেক,লনডন মতিনিধি সংসদ নির্বাচনে জয় লাভ প্রধান মন্...,0
3,আর বাংলাদেশে ফুন দিব না বানকি মুন,জাতিসংঘ মতিবেদক ভবিষ্যতে বাংলাদেশে টেলিফোন অংগ...,0
4,ফোন না করায় জাতিসংঘের মহাসচিবের প্রতি ক্ষুব্ধ...,নিজস্ব মতিবেদক বাকশালের মহিলা আমীর প্রধান মন্ত...,0
...,...,...,...
5575,বেনাপোলের ঘিবা সীমান্তে অস্ত্র ব্যবসায়ী আটক,যশোরের বেনাপোল সীমান্তের ঘিবা গ্রামের বাড়িতে অ...,1
5576,হাতির আক্রমণে সাবেক ছাত্রদল সভাপতির মৃত্যু,মৌলভীবাজার মৌলভীবাজারের জুড়ি উপজেলায় বন্য হাতি...,1
5577,অবরোধের পর ঢাকাচট্টগ্রাম মহাসড়কে যান চলাচল শুরু,নারায়ণগঞ্জের সোনারগাঁওয়ে বাসচাপায় শিক্ষার্থী আ...,1
5578,শ্রমিকদের অবরোধে ঢাকাময়মনসিংহ মহাসড়কে তীব্র যানজট,বকেয়া বেতনের দাবিতে রোববার সকালে গাজীপুর মহানগ...,1


In [294]:
df.groupby('label').agg('count')

,headline,content
label,,
0,2790,2790
1,2789,2789


# CNN

In [295]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from tensorflow.keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from tensorflow.keras.utils import to_categorical
from keras.callbacks import EarlyStopping
%matplotlib inline

In [296]:
X = df.content
Y = df.label
le = LabelEncoder()
Y = le.fit_transform(Y)
Y = Y.reshape(-1,1)

In [297]:
print(Y)

[[0]
 [0]
 [0]
 ...
 [1]
 [1]
 [1]]


In [298]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.25, random_state=1)

In [299]:
# fit a tokenizer
def create_tokenizer(lines):
	tokenizer = Tokenizer()
	tokenizer.fit_on_texts(lines)
	return tokenizer

In [300]:
  # calculate the maximum document length
def max_length(lines):
	return max([len(s.split()) for s in lines])

In [301]:
# calculate vocabulary size
vocab_size = 89000

In [302]:
# encode a list of lines
def encode_text(tokenizer, lines, length):
	# integer encode
	encoded = tokenizer.texts_to_sequences(lines)
	# pad encoded sequences
	padded = pad_sequences(encoded, maxlen=length, padding='post')
	return padded

In [303]:

from pickle import load
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.vis_utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.merge import concatenate
 

In [304]:
# define the model
def define_model(length, vocab_size):
	# channel 1
	inputs1 = Input(shape=(length,))
	embedding1 = Embedding(vocab_size, 100)(inputs1)
	conv1 = Conv1D(filters=32, kernel_size=4, activation='relu')(embedding1)
	drop1 = Dropout(0.5)(conv1)
	pool1 = MaxPooling1D(pool_size=2)(drop1)
	flat1 = Flatten()(pool1)
	# channel 2
	inputs2 = Input(shape=(length,))
	embedding2 = Embedding(vocab_size, 100)(inputs2)
	conv2 = Conv1D(filters=32, kernel_size=6, activation='relu')(embedding2)
	drop2 = Dropout(0.5)(conv2)
	pool2 = MaxPooling1D(pool_size=2)(drop2)
	flat2 = Flatten()(pool2)
	# channel 3
	inputs3 = Input(shape=(length,))
	embedding3 = Embedding(vocab_size, 100)(inputs3)
	conv3 = Conv1D(filters=32, kernel_size=8, activation='relu')(embedding3)
	drop3 = Dropout(0.5)(conv3)
	pool3 = MaxPooling1D(pool_size=2)(drop3)
	flat3 = Flatten()(pool3)
	# merge
	merged = concatenate([flat1, flat2, flat3])
	# interpretation
	dense1 = Dense(10, activation='relu')(merged)
	outputs = Dense(1, activation='sigmoid')(dense1)
	model = Model(inputs=[inputs1, inputs2, inputs3], outputs=outputs)
	# compile
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	# summarize
	print(model.summary())
	plot_model(model, show_shapes=True, to_file='multichannel.png')
	return model

In [305]:
tokenizer = create_tokenizer(X_train)
# calculate max document length
length = max_length(X_train)
# calculate vocabulary size
vocab_size = len(tokenizer.word_index) + 1
print('Max document length: %d' % length)
print('Vocabulary size: %d' % vocab_size)
# encode data
trainX = encode_text(tokenizer, X_train, length)
print(trainX.shape)
 
# define model
model = define_model(length, vocab_size)
# fit model
model.fit([trainX,trainX,trainX], array(Y_train), epochs=10, batch_size=16)
# save the model
model.save('model.h5')

Max document length: 1861
Vocabulary size: 77224
(4184, 1861)
Model: "model_9"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_28 (InputLayer)          [(None, 1861)]       0           []                               
                                                                                                  
 input_29 (InputLayer)          [(None, 1861)]       0           []                               
                                                                                                  
 input_30 (InputLayer)          [(None, 1861)]       0           []                               
                                                                                                  
 embedding_27 (Embedding)       (None, 1861, 100)    7722400     ['input_28[0][0]']               
                              

In [306]:
type(Y_test)

numpy.ndarray

In [318]:
loss, acc = model.evaluate([trainX,trainX,trainX], array(Y_train), verbose=0)
print('Train Accuracy: %f' % (acc*100))

testX = encode_text(tokenizer, X_test, length)

loss, acc = model.evaluate([testX,testX,testX],array(Y_test), verbose=0)
print('Test Accuracy: %f' % (acc*100))

Train Accuracy: 99.976099
Test Accuracy: 96.702510


In [319]:
testX

array([[  130,   144,  2317, ...,     0,     0,     0],
       [ 1291, 15562,    20, ...,     0,     0,     0],
       [ 2188,  7446,   729, ...,     0,     0,     0],
       ...,
       [  412,  1203,   395, ...,     0,     0,     0],
       [   33,  2415,  2869, ...,     0,     0,     0],
       [53789,  3721,  6871, ...,     0,     0,     0]], dtype=int32)

In [320]:
trainX

array([[ 1161,  1417,   343, ...,     0,     0,     0],
       [  326,    97,   371, ...,     0,     0,     0],
       [  130,   144, 29143, ...,     0,     0,     0],
       ...,
       [15942,    28,    67, ...,     0,     0,     0],
       [  457,   144,   103, ...,     0,     0,     0],
       [ 1416,   255,  1385, ...,     0,     0,     0]], dtype=int32)

In [321]:
CNN_pred_y = model.predict([testX, testX, testX])

In [322]:
from sklearn.metrics import classification_report

In [323]:
classification_report(Y_test, np.around(CNN_pred_y), output_dict=True)

{'0': {'precision': 0.9690140845070423,
  'recall': 0.9662921348314607,
  'f1-score': 0.9676511954992968,
  'support': 712},
 '1': {'precision': 0.964963503649635,
  'recall': 0.9677891654465594,
  'f1-score': 0.966374269005848,
  'support': 683},
 'accuracy': 0.9670250896057347,
 'macro avg': {'precision': 0.9669887940783386,
  'recall': 0.9670406501390101,
  'f1-score': 0.9670127322525723,
  'support': 1395},
 'weighted avg': {'precision': 0.9670308968901182,
  'recall': 0.9670250896057347,
  'f1-score': 0.9670260049652284,
  'support': 1395}}

In [324]:
np.around(CNN_pred_y[20:25])

array([[1.],
       [0.],
       [0.],
       [1.],
       [0.]], dtype=float32)

In [325]:
Y_test[20:25]

array([[1],
       [0],
       [0],
       [1],
       [0]])